In [1]:
import math
import random
import copy
import time

from itertools import combinations
from itertools import permutations
import numpy as np
class Individual:
    def __init__(self,code,startArray):
        self.codeWeight=10
        self.breakPointsWeight=50
        self.code=code
        self.fix()
        self.array = []
        self.startArray = startArray
        self.reverse_array()
        self.breakpoints=self.get_break_points()
        self.fitness=self.calculate_fitnes()
        
    def reverse_array(self):
        self.array = copy.deepcopy(self.startArray)
        for k in range(0,len(self.code),2):
            i = self.code[k]
            j = self.code[k+1]
            for l in range(int((j-i+1)/2)):
                self.array[i+l],self.array[j-l] = self.array[j-l],self.array[i+l]



    def get_break_points(self):
        breakPoints=0
        for i in range(1,len(self.array)):
            if abs(self.array[i-1]-self.array[i])>1:
                breakPoints+=1
        return breakPoints
    
    def fix(self):
        
        pom=[]
        for i in range(len(self.code)):
            if i%2==1:
                if self.code[i]<self.code[i-1]:
                    pom.append(self.code[i])
                    pom.append(self.code[i-1])
                elif self.code[i]>self.code[i-1]:
                    pom.append(self.code[i-1])
                    pom.append(self.code[i])
         
        if len(pom)>=2:
            final=[pom[0],pom[1]]
            segment=[pom[0],pom[1]]
            for i in range(2,len(pom)):
                if i%2==1 and  [pom[i-1],pom[i]]!=segment:
                    final.append(pom[i-1])
                    final.append(pom[i])
                    segment=[pom[i-1],pom[i]]
            self.code=final
                
        else :
            self.code=pom
    
    def calculate_fitnes(self):
        return self.breakPointsWeight*self.breakpoints+self.codeWeight*len(self.code)
    
class MSR:
    def __init__(self,array,num):
        self.populationSize=math.floor(len(array)*math.log(len(array))) #Velicina populacije je floor(n*log n)
        self.array=array
        self.breakPoints=self.get_break_points()
        self.population=[]
        self.initialize_population()
        self.tournamentSize=math.ceil(0.15*self.populationSize)
        self.bestIndividual = copy.deepcopy(self.population[0])
        self.mutationRate = 0.35
        self.numOfIters = 10000
        self.populationSelectionPercentege=0.2
        self.num=num
      
    def get_break_points(self):
        breakPoints=0
        for i in range(1,len(self.array)-1):
            if abs(self.array[i-1]-self.array[i])>1:
                breakPoints+=1
        return breakPoints
    
    
    def generate_individual(self,size):
        code=[]
        for i in range(size):
            m=math.ceil(random.randrange(1,math.ceil(len(self.array)/10)+1))
            n=math.ceil(random.randrange(0,math.ceil(9*len(self.array)/10-1)))
            begin=n
            end=m+n
            code.append(begin)
            code.append(end)
        return Individual(code,self.array)
            
    def initialize_population(self):
        lb=self.breakPoints
        ub=3*len(self.array)
        for i in range(self.populationSize):
            size=random.randrange(lb,ub+1)
            self.population.append(self.generate_individual(size))
            
    def selection(self,population):
        
        selected=random.sample(population,max(round(len(population)*(self.tournamentSize/self.populationSize)),1))

        best=min(selected,key=lambda x:x.fitness)
        
        return best
    
    def crossover(self, parent1, parent2):
        if not parent1  or not parent2:
            return Individual([], self.array), Individual([], self.array)
        
          

        if len(parent1) == len(parent2):
            while True:
                firstPoint = random.randrange(0, len(parent1))
                secondPoint = random.randrange(0, len(parent1))
                if firstPoint != secondPoint:
                    break
            if firstPoint > secondPoint:
                firstPoint, secondPoint = secondPoint, firstPoint
            child1 = parent1[:firstPoint] + parent2[firstPoint:secondPoint] + parent1[secondPoint:]
            child2 = parent2[:firstPoint] + parent1[firstPoint:secondPoint] + parent2[secondPoint:]
        else:
            if len(parent1) < len(parent2):
                parent1, parent2 = parent2, parent1
            point = random.randrange(0, len(parent1) - len(parent2) + 1)
            child1 = parent1[:point] + parent2 + parent1[point + len(parent2):]
            child2 = parent1[point:point + len(parent2)]

        return Individual(child1, self.array), Individual(child2, self.array)

    def mutation1(self,individual):
        while(True):
            i = random.randrange(0,int(len(individual.code)/2))
            j = random.randrange(0,int(len(individual.code)/2))
            if i !=j: 
                break
        if j < i :
            i,j = j,i
        i = 2*i
        j = 2*j
        for k in range(j,len(individual.code)-1):
            individual.code[k] = individual.code[k+1]
        del individual.code[-1]
        for k in range(i,len(individual.code)-1):
            individual.code[k] = individual.code[k+1]
        del individual.code[-1]
        
        individual.fix()
        individual.reverse_array()
        individual.breakpoints =individual.get_break_points()
        individual.fitness = individual.calculate_fitnes()

   
    def mutation2(self,individual):
        while(True):
            i = random.randrange(0,int(len(individual.code)/2))
            j = random.randrange(0,int(len(individual.code)/2))
            if i !=j:
                break
        if j < i :
            i,j = j,i
        i = 2*i
        j = 2*j
        while(True):
            value1=random.randrange(0,len(individual.array))
            value2=random.randrange(0,len(individual.array))
            if value1!=value2:
                break
        individual.code.insert(i,value1)
        individual.code.insert(j,value2)
        individual.fix()
        individual.reverse_array()
        individual.breakpoints = individual.get_break_points()
        individual.fitness = individual.calculate_fitnes()

    def mutation3(self,individual): 
        
        if len(individual.code) > 0:
            i = random.randrange(0,len(individual.code))
            individual.code[i] = random.randrange(0,len(individual.array))
            individual.fix()
            individual.reverse_array()
            individual.breakpoints = individual.get_break_points()
            individual.fitness = individual.calculate_fitnes()
        
    def mutation4(self,individual): 
        if len(individual.code)>=2:
            [i,j]=random.sample(range(len(individual.code)),2)
            [val1,val2]=random.sample(range(len(individual.array)),2)
            individual.code[i] = val1
            individual.code[j] = val2
            individual.fix()
            individual.reverse_array()
            individual.breakpoints = individual.get_break_points()
            individual.fitness = individual.calculate_fitnes()
        
    
    def mutate1(self,individual):
        choice=0
        if random.random()< self.mutationRate:
            choice=random.randrange(1,5)
        if choice==1 and len(individual.code)>2:
                self.mutation1(individual)
        if choice==2 and len(individual.code)>2:
                self.mutation2(individual)
        if choice==3:
                self.mutation3(individual)
        if choice==4:
                self.mutation4(individual)
                
                
    def mutate2(self,individual):
        choice=0
        if random.random()< self.mutationRate:
            choice=random.randrange(1,4)
        if choice==1 and len(individual.code)>2:
                self.mutation2(individual)
        if choice==2:
                self.mutation3(individual)
        if choice==3:
                self.mutation4(individual)
                
    def mutate3(self,unit):
        choice=0
        if random.random()< self.mutationRate:
            choice=random.randrange(1,2)
        if choice==1 and len(unit.code)>2:
                self.mutation2(unit)
#         if choice==2:
#                 self.mutation3(unit)
        
       
        
 
    def generate_population(self):
        population=[]
        sorted(self.population,key=lambda x: x.fitness)
        sizeGroup1=2*(math.floor(self.populationSelectionPercentege*self.populationSize)//2)
        group1=self.population[:sizeGroup1]
        group2=self.population[sizeGroup1:]
        for i in range(0,int(sizeGroup1),2):
            parent1 = self.selection(group1)
            parent2 = self.selection(group1)
            child1,child2 = self.crossover(parent1.code,parent2.code)
            temp=[child1,child2,parent1,parent2]
            sorted(temp,key=lambda x:x.fitness)
            child1=copy.deepcopy(temp[0])
            child2=copy.deepcopy(temp[1])
            if self.num==1:
                self.mutate1(child1)
                self.mutate1(child2)
            elif self.num==2:
                self.mutate2(child1)
                self.mutate2(child2)
            else :
                self.mutate3(child1)
                self.mutate3(child2)
                
            population.append(child1)
            population.append(child2)
        while len(population)<self.populationSize:
            parent1 = self.selection(group2)
            parent2 = self.selection(group2)
            child1,child2 = self.crossover(parent1.code,parent2.code)
            temp=[child1,child2,parent1,parent2]
            sorted(temp,key=lambda x:x.fitness)
            child1=copy.deepcopy(temp[0])
            child2=copy.deepcopy(temp[1])
            if self.num==1:
                self.mutate1(child1)
                self.mutate1(child2)
            elif self.num==2:
                self.mutate2(child1)
                self.mutate2(child2)
            else :
                self.mutate3(child1)
                self.mutate3(child2)
            population.append(child1)
            population.append(child2)
        self.population=copy.deepcopy(population)
            
            
        
    def best_solution(self):
        best=min(self.population,key=lambda x:x.fitness)
        return copy.deepcopy(best)

    def solve(self,t):
        fitnessRepetition=0
        oldFitness=0
        start_time = time.time()
        elapsed_time = 0
        duration_limit = t*60
        self.initialize_population()
        while elapsed_time<duration_limit:
            best = self.best_solution()
            fixed_best=Individual(best.code,self.array)
            best=fixed_best
            if best.fitness < self.bestIndividual.fitness:
                self.bestIndividual = copy.deepcopy(best)
            if oldFitness==best.fitness:
                fitnessRepetition+=1
            oldFitness=best.fitness
            
            self.generate_population()
            elapsed_time=time.time()-start_time
            
        return self.bestIndividual




def genetic_algorithm(n,array,k,t):    
    
    bestFitness=float('inf')

    
    

    start=copy.deepcopy(array)
    code=[]
   
    while bestFitness>25:
        
            msr=MSR(array,k)
            individual=msr.solve(t)
            bestFitness=individual.fitness
            new_array=individual.array
            code.append(individual.code)
            array=new_array
    
    breakPoint = -1
    for i in range(1,len(start)):
        if abs(new_array[i-1]-new_array[i])>1:
            breakPoint = i

            
    if breakPoint != -1:
        if abs(new_array[0] - new_array[breakPoint]) == 1: 
            for l in range(int((breakPoint)/2)):
                new_array[l],new_array[breakPoint-1-l] = new_array[breakPoint-1-l],new_array[l]
            code.append([0])
            code.append([breakPoint-1])

        elif abs(new_array[breakPoint-1] - new_array[-1]) == 1: 
            for l in range(int((len(new_array)-breakPoint)/2)):
                new_array[breakPoint+l],new_array[len(new_array)-1-l] = new_array[len(new_array)-1-l],new_array[breakPoint+l]
            code.append([breakPoint])
            code.append([len(new_array)-1])

        else: 
            for l in range(int((breakPoint)/2)):
                new_array[l],new_array[breakPoint-1-l] = new_array[breakPoint-1-l],new_array[l]
            for l in range(int((len(new_array)-breakPoint)/2)):
                new_array[breakPoint+l],new_array[len(new_array)-1-l] = new_array[len(new_array)-1-l],new_array[breakPoint+l]
            code.append([0])
            code.append([breakPoint-1])
            code.append([breakPoint])
            code.append([len(new_array)-1])

    if new_array[0] > new_array[1]:
        for i in range(int(len(new_array)/2)):
            new_array[i],new_array[len(new_array)-1-i] = new_array[len(new_array)-1-i],new_array[i]
        code.append([0])
        code.append([len(new_array)-1])


    pom=[item for sublist in code for item in sublist]
            
  

    ind = Individual(pom,start)

    return str("Kraj:" + " " + "Start Array: " + str(start) + " " + "Najbolja jedinka" + str(ind.code)+ " " +"Broj inverzija: " + str(len(ind.code)/2)+ " " + "Sortirani niz:" + str(ind.array) + " " + "Vrednost fitnes funkcije: " + str(ind.fitness) + "\n")


genetic_algorithm(15,[4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 12 , 15, 14, 13],1,1)



'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 12, 15, 14, 13] Najbolja jedinka[3, 4, 7, 8, 12, 14, 1, 3, 0, 3] Broj inverzija: 5.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] Vrednost fitnes funkcije: 100\n'

In [2]:
genetic_algorithm(15,[4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 12 , 15, 14, 13],2,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 12, 15, 14, 13] Najbolja jedinka[0, 3, 7, 8, 0, 2, 2, 4, 12, 14] Broj inverzija: 5.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] Vrednost fitnes funkcije: 100\n'

In [3]:
genetic_algorithm(15,[4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 12 , 15, 14, 13],3,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 12, 15, 14, 13] Najbolja jedinka[7, 8, 4, 11, 1, 11, 0, 8, 7, 9, 12, 14, 0, 11] Broj inverzija: 7.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] Vrednost fitnes funkcije: 140\n'

In [4]:
genetic_algorithm(14,[4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 14, 12, 13],1,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 14, 12, 13] Najbolja jedinka[1, 6, 1, 3, 11, 12, 12, 13, 0, 13, 5, 9, 6, 11, 5, 9, 10, 13, 0, 13] Broj inverzija: 10.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] Vrednost fitnes funkcije: 200\n'

In [5]:
genetic_algorithm(14,[4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 14, 12, 13],2,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 14, 12, 13] Najbolja jedinka[7, 8, 3, 4, 12, 13, 1, 3, 0, 3, 11, 13] Broj inverzija: 6.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] Vrednost fitnes funkcije: 120\n'

In [6]:
genetic_algorithm(14,[4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 14, 12, 13],3,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 9, 8, 10, 11, 14, 12, 13] Najbolja jedinka[8, 10, 4, 5, 8, 11, 8, 9, 5, 6, 3, 6, 1, 6, 0, 3, 7, 9, 8, 13, 1, 13, 8, 13, 0, 7, 0, 13] Broj inverzija: 14.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] Vrednost fitnes funkcije: 280\n'

In [7]:
genetic_algorithm(13,[4, 1, 2, 5, 3, 6, 7, 10, 8, 9, 11, 12, 13],1,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 10, 8, 9, 11, 12, 13] Najbolja jedinka[3, 4, 1, 12, 4, 5, 4, 6, 1, 9, 0, 9, 0, 12] Broj inverzija: 7.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] Vrednost fitnes funkcije: 140\n'

In [8]:
genetic_algorithm(13,[4, 1, 2, 5, 3, 6, 7, 10, 8, 9, 11, 12, 13],2,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 10, 8, 9, 11, 12, 13] Najbolja jedinka[7, 8, 8, 9, 3, 4, 1, 3, 0, 3] Broj inverzija: 5.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] Vrednost fitnes funkcije: 100\n'

In [9]:
genetic_algorithm(13,[4, 1, 2, 5, 3, 6, 7, 10, 8, 9, 11, 12, 13],3,1)

'Kraj: Start Array: [4, 1, 2, 5, 3, 6, 7, 10, 8, 9, 11, 12, 13] Najbolja jedinka[3, 4, 7, 8, 8, 9, 0, 3, 0, 2] Broj inverzija: 5.0 Sortirani niz:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] Vrednost fitnes funkcije: 100\n'

In [ ]:
33